In [15]:
import pandas as pd
from shapely.geometry import Point
import numpy as np

In [16]:
 
def make_df_subsets(df, line):

    debDf = df[df.HorizonName == 'Mdebolt_interp']
    belDf = df[df.HorizonName == 'PRbelloy_interp']
    wabDf = df[df.HorizonName == 'Wabamun1']

    debDfSubset = debDf[debDf.LineNum == line]
    belDfSubset = belDf[belDf.LineNum == line]
    wabDfSubset = wabDf[wabDf.LineNum == line]

    return debDfSubset, belDfSubset, wabDfSubset


def find_closest_pick(debDf, belDf, wabDf):

    tres = 100
    tracesDeb = []
    tracesBel = []
    tracesWab = []

    XDeb_list = []
    YDeb_list = []
    TimeResDeb_list = []
    XBel_list = []
    YBel_list = []
    TimeResBel_list = []
    XWab_list = []
    YWab_list = []
    TimeResWab_list = []

    lines = []

    for indexDeb, rowDeb in debDf.iterrows():

        traceDistBel = np.abs(rowDeb.TraceNr - belDf.TraceNr)
        traceDistWab = np.abs(rowDeb.TraceNr - wabDf.TraceNr)

        traceDeb = rowDeb.TopDepthResidual

        XDeb = rowDeb.Easting
        YDeb = rowDeb.Northing
        TimeResDeb = rowDeb.TopDepthResidual
    
        if traceDistBel.min() > tres:
            traceBel = np.nan
            XBel, YBel, TimeResBel = np.nan, np.nan, np.nan
        else:
            traceBel = int(resPicks.iloc[traceDistBel.idxmin()]['TraceNr'])
            XBel = resPicks.iloc[traceDistBel.idxmin()]['Easting']
            YBel = resPicks.iloc[traceDistBel.idxmin()]['Northing']
            TimeResBel = resPicks.iloc[traceDistBel.idxmin()]['TopDepthResidual']
        
        if traceDistWab.min() > tres:
            traceWab = np.nan
            XWab, YWab, TimeResWab = np.nan, np.nan, np.nan
        else:
            traceWab = int(resPicks.iloc[traceDistWab.idxmin()]['TraceNr'])
            XWab = resPicks.iloc[traceDistWab.idxmin()]['Easting']
            YWab = resPicks.iloc[traceDistWab.idxmin()]['Northing']
            TimeResWab = resPicks.iloc[traceDistWab.idxmin()]['TopDepthResidual']
            
        tracesDeb.append(traceDeb)
        XDeb_list.append(XDeb)
        YDeb_list.append(YDeb)
        TimeResDeb_list.append(TimeResDeb)
        
        tracesBel.append(traceBel)
        XBel_list.append(XBel)
        YBel_list.append(YBel)
        TimeResBel_list.append(TimeResBel)
        
        tracesWab.append(traceWab)
        XWab_list.append(XWab)
        YWab_list.append(YWab)
        TimeResWab_list.append(TimeResWab)
        
        lines.append(debDf.LineNum.unique()[0])
        
    dfClosestPicks = pd.DataFrame({
        'debPickTrace':tracesDeb,
        'belPickTrace':tracesBel,
        'wabPickTrace':tracesWab,
        'debPickX':XDeb_list,
        'debPickY':YDeb_list,
        'debRes':TimeResDeb_list,
        'belPickX':XBel_list,
        'belPickY':YBel_list,
        'belRes':TimeResBel_list,
        'wabPickX':XWab_list,
        'wabPickY':YWab_list,
        'wabRes':TimeResWab_list,
        'line':lines})
    #            'noPicks':sum(pd.notna([debTraces, belTraces, wabTraces])),
    return dfClosestPicks

def calc_dip(df):
    dips = []

    df = df.reset_index()
    for idx, row in df.iterrows():
        
        res_list = row[['belRes','debRes','wabRes']].tolist()
        X_list = row[['belPickX','debPickX','wabPickX']].tolist()
        Y_list = row[['belPickY','debPickY','wabPickY']].tolist()
        
        if res_list.count('NaN') < 2:
            idxmin = res_list.index(np.nanmin(res_list))
            idxmax = res_list.index(np.nanmax(res_list))
            res0 = res_list[idxmin]
            res1 = res_list[idxmax]
            P0 = Point(X_list[idxmin],Y_list[idxmin])
            P1 = Point(X_list[idxmax],Y_list[idxmax])
            
            dx = P0.distance(P1)
            dt = (res1 - res0)*1000
        if ((dx == 0) & (dt == 0)):
            dip = 0
        else:
            dip = dx / dt
                
        dips.append(dip)
    df['dip'] = dips
    return df

In [17]:
resPicks = pd.read_csv('/home/paulina/Desktop/NRCan/scripts/data/faultPicks_1stOrd.txt', delimiter='\t')
resPicks.head()

,Easting,Northing,TopDepth,TopDepthPred,TopDepthResidual,TraceNr,LineNum,HorizonName
0,281418,6201553.33,1.7747,1.848842,-0.074142,240,L11a_smooth,Mdebolt_interp
1,294715,6199393.04,1.7769,1.812971,-0.036071,1323,L11a_smooth,Mdebolt_interp
2,297749,6199250.16,1.7660,1.803793,-0.037793,1566,L11a_smooth,Mdebolt_interp
3,300359,6199126.33,1.7697,1.795901,-0.026201,1775,L11a_smooth,Mdebolt_interp
4,301482,6199080.00,1.7649,1.792485,-0.027585,1865,L11a_smooth,Mdebolt_interp


In [18]:
picksDf = pd.DataFrame(columns={
        'debPickTrace':[],
        'belPickTrace':[],
        'wabPickTrace':[],
        'debPickX':[],
        'debPickY':[],
        'debRes':[],
        'belPickX':[],
        'belPickY':[],
        'belRes':[],
        'wabPickX':[],
        'wabPickY':[],
        'wabRes':[],
        'line':[]})
 
for line in resPicks.LineNum.unique():
    debDfSubset, belDfSubset, wabDfSubset = make_df_subsets(resPicks, line)
    df = find_closest_pick(debDfSubset, belDfSubset, wabDfSubset)
    df2 = calc_dip(df)

    picksDf = picksDf.append(df, sort=False)
    
print(df.head())
print(picksDf.shape)


   debPickTrace  belPickTrace  wabPickTrace  debPickX    debPickY    debRes  \
0     -0.017291         374.0           NaN    503235  6149149.92 -0.017291   
1     -0.029385         551.0         566.0    504344  6149151.01 -0.029385   
2     -0.034740         551.0         566.0    505118  6149151.05 -0.034740   
3     -0.024391         551.0         624.0    506304  6149152.14 -0.024391   
4     -0.030012         738.0         777.0    507467  6149153.90 -0.030012   

   belPickX    belPickY    belRes  wabPickX    wabPickY    wabRes      line  
0  502986.0  6149150.00 -0.019267       NaN         NaN       NaN  L14ab(1)  
1  505193.0  6149151.14 -0.023002  505380.0  6149151.44 -0.021926  L14ab(1)  
2  505193.0  6149151.14 -0.023002  505380.0  6149151.44 -0.021926  L14ab(1)  
3  505193.0  6149151.14 -0.023002  506104.0  6149151.94 -0.021674  L14ab(1)  
4  507529.0  6149154.05 -0.016696  508017.0  6149155.39 -0.021152  L14ab(1)  
(117, 13)


In [19]:
picksDf['noPicks'] = picksDf.apply(lambda x: sum(pd.notna(x[['debPickTrace','belPickTrace','wabPickTrace']])), 
                                   axis=1)
picksDf.head()

,debPickTrace,belPickTrace,wabPickTrace,debPickX,debPickY,debRes,belPickX,belPickY,belRes,wabPickX,wabPickY,wabRes,line,noPicks
0,-0.074142,271.0,NaN,281418,6201553.33,-0.074142,281803.0,6201548.80,-0.083527,NaN,NaN,NaN,L11a_smooth,2
1,-0.036071,NaN,1352.0,294715,6199393.04,-0.036071,NaN,NaN,NaN,295077.0,6199374.94,0.007761,L11a_smooth,2
2,-0.037793,1579.0,1531.0,297749,6199250.16,-0.037793,297911.0,6199242.66,-0.026307,297312.0,6199269.83,-0.012718,L11a_smooth,3
3,-0.026201,NaN,1798.0,300359,6199126.33,-0.026201,NaN,NaN,NaN,300646.0,6199119.83,-0.000730,L11a_smooth,2
4,-0.027585,NaN,1798.0,301482,6199080.00,-0.027585,NaN,NaN,NaN,300646.0,6199119.83,-0.000730,L11a_smooth,2


In [20]:
 
dfDips = calc_dip(picksDf)
dfDips.head()

,index,debPickTrace,belPickTrace,wabPickTrace,debPickX,debPickY,debRes,belPickX,belPickY,belRes,wabPickX,wabPickY,wabRes,line,noPicks,dip
0,0,-0.074142,271.0,NaN,281418,6201553.33,-0.074142,281803.0,6201548.80,-0.083527,NaN,NaN,NaN,L11a_smooth,2,41.023971
1,1,-0.036071,NaN,1352.0,294715,6199393.04,-0.036071,NaN,NaN,NaN,295077.0,6199374.94,0.007761,L11a_smooth,2,8.269009
2,2,-0.037793,1579.0,1531.0,297749,6199250.16,-0.037793,297911.0,6199242.66,-0.026307,297312.0,6199269.83,-0.012718,L11a_smooth,3,17.444767
3,3,-0.026201,NaN,1798.0,300359,6199126.33,-0.026201,NaN,NaN,NaN,300646.0,6199119.83,-0.000730,L11a_smooth,2,11.270854
4,4,-0.027585,NaN,1798.0,301482,6199080.00,-0.027585,NaN,NaN,NaN,300646.0,6199119.83,-0.000730,L11a_smooth,2,31.165688


In [21]:
# NOT ENCODED
from sklearn.preprocessing import MinMaxScaler

dfML = dfDips[['debPickTrace', 'debPickX', 'debPickY','dip','noPicks']]    
X = dfML.values
X_scaled = MinMaxScaler().fit_transform(X)

model_kmean = KMeans(n_clusters=15,n_init=8, random_state=1234)
y_pred_kmean = model_kmean.fit_predict(X_scaled)
y_pred_kmean

kmean_params_dict = model_kmean.get_params()
keys = ['n_clusters']
kmean_params = [(key, kmean_params_dict[key]) for key in keys]
dfML_kmean = dfML.copy()
dfML_kmean['label'] = y_pred_kmean.tolist()
 
print(dfML_kmean['label'].unique())

[ 7  8  0 11  2 10  5  6  4 14 13  9 12  1  3]


In [22]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn import preprocessing
import random # for color codes generation
dfML = dfDips[['debPickTrace', 'debPickX', 'debPickY','dip','noPicks']]    

# dfML = dfDips[['debPickTrace','dip','noPicks']]    
X = dfML.values
X_scaled = MinMaxScaler().fit_transform(X)

model_db = DBSCAN(eps=0.13, metric='euclidean', min_samples=3) # metric='euclidean'
y_pred_dbscan = model_db.fit_predict(X_scaled)
# y_pred_dbscan

db_params_dict = model_db.get_params()
keys = ['eps','metric','min_samples']
db_params = [(key, db_params_dict[key]) for key in keys]
dfML_dbscan = dfML.copy()
dfML_dbscan['label'] = y_pred_dbscan.tolist()

In [23]:
fig = go.Figure(
layout = go.Layout(title="Reclustering results - indicated by the Debolt pick on the fault",
                   xaxis= dict(title= 'World Rank')))

np.random.RandomState(seed=1234)
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
cluster_labels = dfML_dbscan.label.unique().tolist()
label_colors_list = get_colors(len(cluster_labels))

for label in dfML_dbscan.label.unique():
    df_group = dfML_dbscan[dfML_dbscan['label'] == label]
    group_label = df_group.label.unique()
    
    #assign color index to show faults on the same line with one color
    label_idx = cluster_labels.index(group_label)

    x=df_group.debPickX.values
    y=df_group.debPickY.values
    z=df_group.debPickTrace.values
     
    fig.add_trace(go.Scatter3d(
        x=-x,
        y=-y,
        z=-z,
        mode='markers',
        marker=dict(color=label_colors_list[label_idx], size=2),
        legendgroup=label_idx,name='Fault group' + str(cluster_labels[label_idx])))
    
fig.update_scenes(aspectmode='manual',
                  aspectratio=dict(x=2, y=2, z=1))
   
# fig.write_html("/home/paulina/Desktop/for_DAVE/faults_dbscan.html")

fig.show()

In [24]:
# alternative with OHE (in progress)
# X = dfML.copy()
# from sklearn.compose import make_column_transformer
# from sklearn.pipeline import make_pipeline

# column_trans = make_column_transformer((
#     OneHotEncoder(), [['noPicks']]),
#     (StandardScaler(), [['debPickTrace','debPickX','debPickY', 'dip']]),
#     remainder='passthrough')

# columnTransformer = ColumnTransformer([('encoder', 
#                                         OneHotEncoder(), 
#                                         [['noPicks']])], 
#                                       remainder='passthrough') 

# data = np.array(columnTransformer.fit_transform(dfML), dtype = np.str) 

# data = np.array(column_trans.fit_transform(X)) 

# enc = OneHotEncoder(handle_unknown='ignore')
# enc_noPicks = enc.fit(dfML[['noPicks']])
# # le.transform(dfML.noPicks)
# # creating one hot encoder object with categorical feature 0 
# # indicating the first column 
# columnTransformer = ColumnTransformer([('encoder', 
#                                         OneHotEncoder(), 
#                                         '[0]')], 
#                                       remainder='passthrough') 
  
# data = np.array(columnTransformer.fit_transform(data), dtype = np.str) 
# kmean = KMeans(n_clusters=15, n_init=8, random_state=1234)
# pipe = make_pipeline(column_trans, kmean)
# pipe.fit_predict()
# predicted_ohe = pipe.fit_predict(X)